In [1]:
import pandas as pd


In [2]:
cred = {"host": 'localhost', 'dbname': 'yukontaf', 'user': 'glebsokolov', 'password': ''}
from sqlalchemy import create_engine
import zipfile, os

con = create_engine(
    f'postgresql://{cred["user"]}:{cred["password"]}@{cred["host"]}/{cred["dbname"]}'
)


def select(sql, con):
    with open(sql) as f:
        return pd.read_sql(f.read(), con)


def send(name=None, table=None, path=None, sheetn=None, con=con):
    if table is not None and name:
        return table.to_sql(name, if_exists='replace', con=con)
    elif path and name:
        if os.path.basename(path).endswith('.zip'):
            with zipfile.ZipFile(path, 'r') as zip_ref:
                folder = os.path.dirname(path) + '/' + name
                zip_ref.extractall(folder)
                os.remove(path)
            for root, dirs, files in os.walk(folder):
                for file in files:
                    if file.endswith('.csv') or file.endswith('.txt'):
                        if file.endswith('.txt'): sep = ' '
                        if name != 'zip':
                            pd.read_csv(os.path.join(root, file)).to_sql(name + '.' + file[:-4], if_exists='replace',
                                                                         con=con)
                        else:
                            pd.read_csv(os.path.join(root, file), sep=sep).to_sql(file[:-4], if_exists='replace',
                                                                                  con=con)
                    elif file.endswith('.json'):
                        pd.read_json(os.path.join(root, file)).to_sql(file[:-5], if_exists='replace',
                                                                      con=con)
                    os.remove(os.path.join(root, file))
                return os.rmdir(folder)
        elif os.path.basename(path).endswith('.csv'):
            return pd.read_csv(path, encoding='utf-8', encoding_errors='ignore').to_sql(name, if_exists='replace',
                                                                                        con=con)
        elif os.path.basename(path).endswith('.xlsx'):
            if sheetn:
                return pd.read_excel(path, sheet_name=sheetn).to_sql(name, if_exists='replace', con=con)
            else:
                sheetn = input(f'Choose Sheet:{pd.ExcelFile(path).sheet_names}')
                return pd.read_excel(path, sheet_name=sheetn).to_sql(name, if_exists='replace', con=con)
    elif path:
        if os.path.basename(path).endswith('.csv') or os.path.basename(path).endswith('.zip'):
            return pd.read_csv(path).to_sql(os.path.basename(path)[:-4], if_exists='replace', con=con)
        elif os.path.basename(path).endswith('.xlsx'):
            if sheetn:
                return pd.read_excel(path, sheet_name=sheetn).to_sql(os.path.basename(path)[:-5], if_exists='replace',
                                                                     con=con)
            else:
                sheetn = input(f'Choose Sheet:{pd.ExcelFile(path).sheet_names}')
                return pd.read_excel(path, sheet_name=sheetn).to_sql(os.path.basename(path)[:-5], if_exists='replace',
                                                                     con=con)



In [4]:
pd.read_csv('/Users/glebsokolov/Downloads/sales_train+.csv', sep=',', encoding_errors='ignore', on_bad_lines='skip')

,D
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1823215,NaN
1823216,NaN
1823217,NaN
1823218,NaN


In [3]:
pd.read_excel('/Users/glebsokolov/Downloads/sales_train+.xlsx', sheet_name=0)

/Users/glebsokolov/.local/share/virtualenvs/glebsokolov-qQcMW7DI/lib/python3.9/site-packages/openpyxl/reader/workbook.py:88: UserWarning: File contains an invalid specification for sales_train+. This will be removed
  warn(msg)


ValueError: Worksheet index 0 is invalid, 0 worksheets found

In [5]:
path = '/Users/glebsokolov/Downloads/supermarket_sales - Sheet1.csv'
name = 'supermarket_sales'
send(path=path, name=name)

1000

In [2]:
import wget

In [3]:
import requests
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

url = 'http://library.lol/main/6393A6CD6994DA538EADEBFE61C1F907'

bs = BeautifulSoup(requests.Session().get(url).text, 'html.parser')
link = bs.find_all(lambda tag: tag.name == 'a' and 'Cloudflare' in tag.text)[0]['href']
ext = '.pdf' if 'pdf' in link else '.epub'
name = bs.find('h1').get_text()
wget.download(link, out='/Users/glebsokolov/Downloads/' + name + ext)

'/Users/glebsokolov/Downloads/Data Science from Scratch: First Principles with Python.epub'

In [2]:
df = pd.read_csv('/Users/glebsokolov/Downloads/P12-Bank-Customers-Demo (1).csv')

In [4]:
df['Date Joined'] = df['Date Joined'].apply(lambda x: pd.to_datetime(x))

In [13]:
df.to_csv('/Users/glebsokolov/Downloads/P12-Bank-Customers-Demo (1).csv')

In [11]:
df['Balance'] = df['Balance'].apply(lambda x: float(x.replace('$', '').replace(',', '')))

In [12]:
df

,Customer ID,Name,Surname,Gender,Age,Date Joined,Balance
0,100000001,Simon,Walsh,Male,21,2015-01-05,113810.15
1,400000002,Jasmine,Miller,Female,34,2015-01-06,36919.73
2,100000003,Liam,Brown,Male,46,2015-01-07,101536.83
3,300000004,Trevor,Parr,Male,32,2015-01-08,1421.52
4,100000005,Deirdre,Pullman,Female,38,2015-01-09,35639.79
5,300000006,Ava,Coleman,Female,30,2015-01-09,122443.77
6,100000007,Dorothy,Thomson,Female,34,2015-01-11,42879.84
7,200000008,Lisa,Knox,Female,48,2015-01-11,36680.17
8,300000009,Ruth,Campbell,Female,33,2015-01-11,74284.35
9,100000010,Dominic,Parr,Male,42,2015-01-12,10912.45
